In [1]:
pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [102]:
import pandas as pd
from collections import defaultdict

In [183]:
#Definição da classe Arvore Binaria com os alimentos

class NoArvore:
    def __init__(self, alimento, calorias_por_grama):
        self.alimento = alimento
        self.calorias_por_grama = calorias_por_grama
        self.esquerda = None
        self.direita = None
        self.altura = 1

class ArvoreBinaria:
    def __init__(self):
        self.raiz = None

    def adicionar(self, alimento, calorias_por_grama):
        if self.raiz is None:
            self.raiz = NoArvore(alimento, calorias_por_grama)
        else:
            self.raiz = self._adicionar_recursivamente(self.raiz, alimento, calorias_por_grama)

    def _altura(self, no):
        if no is None:
            return 0
        return no.altura

    def _fator_balanceamento(self, no):
        if no is None:
            return 0
        return self._altura(no.esquerda) - self._altura(no.direita)

    def _rotacao_direita(self, y):
        x = y.esquerda
        z = x.direita

        x.direita = y
        y.esquerda = z

        y.altura = 1 + max(self._altura(y.esquerda), self._altura(y.direita))
        x.altura = 1 + max(self._altura(x.esquerda), self._altura(x.direita))

        return x

    def _rotacao_esquerda(self, x):
        y = x.direita
        z = y.esquerda

        y.esquerda = x
        x.direita = z

        x.altura = 1 + max(self._altura(x.esquerda), self._altura(x.direita))
        y.altura = 1 + max(self._altura(y.esquerda), self._altura(y.direita))

        return y

    def adicionar(self, alimento, calorias_por_grama):
        if self.raiz is None:
            self.raiz = NoArvore(alimento, calorias_por_grama)
        else:
            self.raiz = self._adicionar_recursivamente(self.raiz, alimento, calorias_por_grama)

    def _adicionar_recursivamente(self, no_atual, alimento, calorias_por_grama):
        if no_atual is None:
            return NoArvore(alimento, calorias_por_grama)

        if alimento < no_atual.alimento:
            no_atual.esquerda = self._adicionar_recursivamente(no_atual.esquerda, alimento, calorias_por_grama)
        elif alimento > no_atual.alimento:
            no_atual.direita = self._adicionar_recursivamente(no_atual.direita, alimento, calorias_por_grama)
        else:
            print(f'O alimento "{alimento}" já existe na árvore.')
            return no_atual

        no_atual.altura = 1 + max(self._altura(no_atual.esquerda), self._altura(no_atual.direita))

        fator = self._fator_balanceamento(no_atual)

        if fator > 1 and alimento < no_atual.esquerda.alimento:
            return self._rotacao_direita(no_atual)

        if fator < -1 and alimento > no_atual.direita.alimento:
            return self._rotacao_esquerda(no_atual)

        if fator > 1 and alimento > no_atual.esquerda.alimento:
            no_atual.esquerda = self._rotacao_esquerda(no_atual.esquerda)
            return self._rotacao_direita(no_atual)

        if fator < -1 and alimento < no_atual.direita.alimento:
            no_atual.direita = self._rotacao_direita(no_atual.direita)
            return self._rotacao_esquerda(no_atual)

        return no_atual
    
    def buscar(self, alimento):
        return self._buscar_recursivamente(alimento, self.raiz)

    def _buscar_recursivamente(self, alimento, no_atual):
        if no_atual is None or no_atual.alimento == alimento:
            return no_atual
        if alimento < no_atual.alimento:
            return self._buscar_recursivamente(alimento, no_atual.esquerda)
        return self._buscar_recursivamente(alimento, no_atual.direita)
    
    def imprimir_arvore(self):
        self._imprimir_recursivamente(self.raiz, 0)

    def _imprimir_recursivamente(self, no_atual, nivel):
        if no_atual is not None:
            self._imprimir_recursivamente(no_atual.esquerda, nivel + 1)
            print("   " * nivel, end="")
            print(f"{no_atual.alimento} ({no_atual.calorias_por_grama})")
            self._imprimir_recursivamente(no_atual.direita, nivel + 1)

    def listar_nos(self):
        todos_nos = []
        self._listar_nos_recursivamente(self.raiz, todos_nos)
        return todos_nos

    def _listar_nos_recursivamente(self, no_atual, todos_nos):
        if no_atual is not None:
            self._listar_nos_recursivamente(no_atual.esquerda, todos_nos)
            todos_nos.append({'alimento': no_atual.alimento, 'calorias_por_grama': no_atual.calorias_por_grama})
            self._listar_nos_recursivamente(no_atual.direita, todos_nos)

    def buscar_calorias_por_grama(self, alimento):
        no = self.buscar(alimento)
        if no:
            return no.calorias_por_grama
        else:
            return None

In [184]:
#Função de busca do alimento na árvore

def buscar_alimento(no_atual, alimento):
    if no_atual is None or no_atual.alimento == alimento:
        return no_atual
    if alimento < no_atual.alimento:
        return buscar_alimento(no_atual.esquerda, alimento)
    return buscar_alimento(no_atual.direita, alimento)

In [185]:
# Criando uma instância da árvore
arvore_alimentos = ArvoreBinaria()


In [186]:
def salvar_arvore_txt(no_atual, arquivo):
    if no_atual is not None:
        if no_atual.direita is not None:
            salvar_arvore_txt(no_atual.direita, arquivo)
        arquivo.write(f"{no_atual.alimento},{no_atual.calorias_por_grama}\n")
        if no_atual.esquerda is not None:
            salvar_arvore_txt(no_atual.esquerda, arquivo)

def salvar_arvore(nome_arquivo, arvore):
    with open(nome_arquivo, 'w') as arquivo_txt:
        if arvore.raiz is not None:
            salvar_arvore_txt(arvore.raiz, arquivo_txt)

In [187]:

def ler_arvore_txt(nome_arquivo):
    arvore = ArvoreBinaria()
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
        for linha in linhas:
            dados = linha.strip().split(',')
            alimento = dados[0]
            calorias_por_grama = float(dados[1])
            arvore.raiz = adicionar_no(arvore.raiz, alimento, calorias_por_grama)
    return arvore

def adicionar_no(raiz, alimento, calorias_por_grama):
    if raiz is None:
        return NoArvore(alimento, calorias_por_grama)
    elif alimento < raiz.alimento:
        raiz.esquerda = adicionar_no(raiz.esquerda, alimento, calorias_por_grama)
    elif alimento > raiz.alimento:
        raiz.direita = adicionar_no(raiz.direita, alimento, calorias_por_grama)
    return raiz



In [188]:
def adicionar_alimento_consumido(arvore, data_consumo, alimento, quantidade_consumida):
    resultado_busca = buscar_alimento(arvore.raiz, alimento)
    
    if resultado_busca:
        # Aqui você pode fazer o cálculo das calorias com base na quantidade consumida
        calorias_consumidas = resultado_busca.calorias_por_grama * quantidade_consumida
        
        if data_consumo not in alimentos_consumidos:
            alimentos_consumidos[data_consumo] = []

        alimentos_consumidos[data_consumo].append({
            'alimento': alimento,
            'quantidade_consumida': quantidade_consumida,
            'calorias_consumidas': calorias_consumidas
        })
        print(f"Alimento '{alimento}' adicionado na data {data_consumo}.")
        return calorias_consumidas
    else:
        print(f"Alimento '{alimento}' não encontrado na lista de alimentos.")
        return 0

def calcular_calorias_por_dia(alimentos_consumidos):
    calorias_por_dia = defaultdict(int)

    for data, consumos in alimentos_consumidos.items():
        for consumo in consumos:
            calorias_por_dia[data] += consumo['calorias_consumidas']

    return calorias_por_dia




In [189]:
alimentos_consumidos = defaultdict(list)

In [190]:
arquivoArvore = "TesteArvore.txt"

In [191]:
arvore_lida = ler_arvore_txt(arquivoArvore)

In [192]:
arvore_lida.imprimir_arvore()

            Abacate (1.6)
         Abacaxi (0.5)
                     Abobrinha (0.2)
                        Aipo (0.2)
                  Alface (0.1)
                        Alho (1.5)
                           Amendoim (5.8)
                     Amêndoas (5.9)
               Arroz branco cozido (1.1)
                     Arroz integral cozido (1.1)
                        Arroz selvagem cozido (1.0)
                  Atum enlatado (1.3)
                     Aveia cozida (0.6)
                        Avelãs (6.1)
            Açúcar branco (4.0)
               Bacalhau (1.8)
      Banana (0.9)
         Batata cozida (0.7)
                     Berinjela (0.2)
                  Beterraba cozida (0.4)
                           Biscoito cream cracker (4.5)
                        Biscoito de polvilho (4.7)
                     Biscoito de água e sal (4.4)
                        Biscoito maisena (4.8)
                           Biscoito recheado (4.6)
               Brócolis cozido (0.3

In [193]:
# Exemplo de uso
adicionar_alimento_consumido(arvore_lida, '2023-11-20', 'Maçã', 50)  # Exemplo de adição de alimento consumido
calorias_por_dia = calcular_calorias_por_dia(alimentos_consumidos)
print("Calorias por dia:")
for data, calorias in calorias_por_dia.items():
    print(f"{data}: {calorias} cal")

Alimento 'Maçã' adicionado na data 2023-11-20.
Calorias por dia:
2023-11-20: 25.0 cal


In [194]:
adicionar_alimento_consumido(arvore_lida, '2023-11-20', 'Maçã', 50)  # Exemplo de adição de alimento consumido
calorias_por_dia = calcular_calorias_por_dia(alimentos_consumidos)
print("Calorias por dia:")
for data, calorias in calorias_por_dia.items():
    print(f"{data}: {calorias} cal")

Alimento 'Maçã' adicionado na data 2023-11-20.
Calorias por dia:
2023-11-20: 50.0 cal


In [195]:
adicionar_alimento_consumido(arvore_lida, '2023-11-21', 'Maçã', 100)  # Exemplo de adição de alimento consumido
calorias_por_dia = calcular_calorias_por_dia(alimentos_consumidos)
print("Calorias por dia:")
for data, calorias in calorias_por_dia.items():
    print(f"{data}: {calorias} cal")

Alimento 'Maçã' adicionado na data 2023-11-21.
Calorias por dia:
2023-11-20: 50.0 cal
2023-11-21: 50.0 cal


In [196]:
class RegistroAlimentosConsumidos:
    def __init__(self):
        self.registro = defaultdict(list)

    def adicionar_consumo(self, arvore, data, alimento, quantidade):
        resultado_busca = arvore.buscar_calorias_por_grama(alimento)
        
        if resultado_busca is not None:
            calorias_consumidas = resultado_busca * quantidade
            
            self.registro[data].append({
                'alimento': alimento,
                'quantidade_consumida': quantidade,
                'calorias_consumidas': calorias_consumidas
            })
            print(f"Alimento '{alimento}' adicionado na data {data}.")
        else:
            print(f"Alimento '{alimento}' não encontrado na lista de alimentos.")

    """
    def listar_alimentos_consumidos(self, data):
        alimentos = self.registro.get(data, [])
        if alimentos:
            print(f"Alimentos consumidos em {data}:")
            for consumo in alimentos:
                print(f"  Alimento: {consumo['alimento']}, Quantidade: {consumo['quantidade_consumida']}, Calorias: {consumo['calorias_consumidas']}")
        else:
            print(f"Nenhum alimento consumido em {data}.")

    def listar_todos_alimentos_consumidos(self):
        if self.registro:
            print("Todos os alimentos consumidos:")
            for data, alimentos in self.registro.items():
                print(f"  Data: {data}")
                for consumo in alimentos:
                    print(f"    Alimento: {consumo['alimento']}, Quantidade: {consumo['quantidade_consumida']}, Calorias: {consumo['calorias_consumidas']}")
        else:
            print("Nenhum alimento consumido.")

    """
    
    def listar_alimentos_consumidos(self, data):
        if data in self.registro:
            return self.registro[data]
        else:
            return []
        

    def listar_todos_alimentos_consumidos(self):
        return self.registro

    def calcular_calorias_todos_dias(self):
        calorias_todos_dias = defaultdict(int)

        for data, consumos in self.registro.items():
            for consumo in consumos:
                calorias_todos_dias[data] += consumo['calorias_consumidas']

        return calorias_todos_dias
    
    def calcular_calorias_por_dia(self, data):
        calorias_por_dia = 0

        for consumo_data, consumos in self.registro.items():
            if consumo_data == data:
                for consumo in consumos:
                    calorias_por_dia += consumo['calorias_consumidas']
                break

        return calorias_por_dia


In [197]:
## Supondo que 'registro' seja uma instância da classe RegistroAlimentosConsumidos
registro = RegistroAlimentosConsumidos()



In [198]:
# Supondo que 'arvore' seja uma instância da árvore de alimentos
arvore = arvore_lida

# Adicionar consumo com a árvore de alimentos
registro.adicionar_consumo(arvore, '2023-11-21', 'Maçã', 200)
registro.adicionar_consumo(arvore, '2023-11-21', 'Manteiga', 2)
registro.adicionar_consumo(arvore, '2023-11-21', 'Arroz', 100)

registro.adicionar_consumo(arvore, '2023-11-22', 'Manteiga', 40)
registro.adicionar_consumo(arvore, '2023-11-22', 'Banana', 400)


Alimento 'Maçã' adicionado na data 2023-11-21.
Alimento 'Manteiga' adicionado na data 2023-11-21.
Alimento 'Arroz' não encontrado na lista de alimentos.
Alimento 'Manteiga' adicionado na data 2023-11-22.
Alimento 'Banana' adicionado na data 2023-11-22.


In [199]:
# Simulando Tela "Today's Foods"

calorias_hoje = registro.calcular_calorias_por_dia('2023-11-21')

print(f"Calorias consumidas hoje: {calorias_hoje} cal")

lista_consumo = registro.listar_todos_alimentos_consumidos()

print("Todos os alimentos consumidos:")
for data, alimentos in lista_consumo.items():
    print(f"  Data: {data}")
    for consumo in alimentos:
        print(f"    Alimento: {consumo['alimento']}, " + 
              f"Quantidade: {consumo['quantidade_consumida']}, " + 
              f"Calorias: {consumo['calorias_consumidas']}")
    print("")

print("Todos os alimentos consumidos em 2023-11-21:")
lista_consumo = registro.listar_alimentos_consumidos('2023-11-21')
for consumo in lista_consumo:
    print(f"    Alimento: {consumo['alimento']}, " + 
          f"Quantidade: {consumo['quantidade_consumida']}, " + 
          f"Calorias: {consumo['calorias_consumidas']}")
print("")


Calorias consumidas hoje: 114.0 cal
Todos os alimentos consumidos:
  Data: 2023-11-21
    Alimento: Maçã, Quantidade: 200, Calorias: 100.0
    Alimento: Manteiga, Quantidade: 2, Calorias: 14.0

  Data: 2023-11-22
    Alimento: Manteiga, Quantidade: 40, Calorias: 280.0
    Alimento: Banana, Quantidade: 400, Calorias: 360.0

Todos os alimentos consumidos em 2023-11-21:
    Alimento: Maçã, Quantidade: 200, Calorias: 100.0
    Alimento: Manteiga, Quantidade: 2, Calorias: 14.0



In [205]:
# Simulando Tela "All Foods"
print("Buscar alimentos na árvore:")
alimento_para_buscar = 'Melancia'
resultado_busca = arvore.buscar(alimento_para_buscar)
if resultado_busca:
    print(f"Nome: {resultado_busca.alimento}, Calorias por grama: {resultado_busca.calorias_por_grama}")
else:
    print(f"Alimento '{alimento_para_buscar}' não encontrado.")

print("\nTodos os alimentos disponíveis:")
alimentos = arvore.listar_nos()
for alimento in alimentos:
    print(f"  Alimento: {alimento['alimento']}, Calorias por grama: {alimento['calorias_por_grama']}")

Buscar alimentos na árvore:
Nome: Melancia, Calorias por grama: 0.3

Todos os alimentos disponíveis:
  Alimento: Abacate, Calorias por grama: 1.6
  Alimento: Abacaxi, Calorias por grama: 0.5
  Alimento: Abobrinha, Calorias por grama: 0.2
  Alimento: Aipo, Calorias por grama: 0.2
  Alimento: Alface, Calorias por grama: 0.1
  Alimento: Alho, Calorias por grama: 1.5
  Alimento: Amendoim, Calorias por grama: 5.8
  Alimento: Amêndoas, Calorias por grama: 5.9
  Alimento: Arroz branco cozido, Calorias por grama: 1.1
  Alimento: Arroz integral cozido, Calorias por grama: 1.1
  Alimento: Arroz selvagem cozido, Calorias por grama: 1.0
  Alimento: Atum enlatado, Calorias por grama: 1.3
  Alimento: Aveia cozida, Calorias por grama: 0.6
  Alimento: Avelãs, Calorias por grama: 6.1
  Alimento: Açúcar branco, Calorias por grama: 4.0
  Alimento: Bacalhau, Calorias por grama: 1.8
  Alimento: Banana, Calorias por grama: 0.9
  Alimento: Batata cozida, Calorias por grama: 0.7
  Alimento: Berinjela, Caloria

In [215]:
# Simular tela "Calendário"

print("\nCalendário de Novembro:")
for dia in range(1, 31 + 1):
    day = f"{dia:2}" if dia != 21 and dia != 22 else f"\033[1;31m{dia:2}\033[0;0m"
    endingCaracter = " " if dia % 7 else "\n"
    print(day, end=endingCaracter)

calorias_hoje = registro.calcular_calorias_por_dia('2023-11-22')
print(f"\nCalorias ingeridas hoje: {calorias_hoje} cal")


Calendário de Novembro:
 1  2  3  4  5  6  7
 8  9 10 11 12 13 14
15 16 17 18 19 20 21
22 23 24 25 26 27 28
29 30 31 
Calorias ingeridas hoje: 640.0 cal
